In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
import keras
print(keras.__version__)

3.8.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL/11.project3/title_categorizer/data/torob_clean_train_3.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL/11.project3/title_categorizer/data/torob_clean_test_3.csv')

In [5]:
train_data

,name1,name2,cat_id,clean_name
0,کاپشن کوهنوردی پر سنگین نورث فیس,NaN,0,کاپشن کوهنوردی سنگین نورث فیس
1,پالتو خزدار مردانه مارک اصلی,NaN,0,پالتو خزدار مردانه مارک اصلی
2,کاپشن مردانه مدل Bako,NaN,0,کاپشن مردانه مدل Bako
3,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,NaN,0,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...
4,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,NaN,0,کاپشن نظامی آمریکایی سبز
...,...,...,...,...
8817,ساک پارچه ای I have spoken,NaN,10,ساک پارچه ای I have spoken
8818,ساک دستی بیسراک Bisrak کد 1172,NaN,10,ساک دستی بیسراک Bisrak کد
8819,ساک طرحدار پارچه ای,NaN,10,ساک طرحدار پارچه ای
8820,کیف باشگاه پوما ا puma | 31349075,puma | 31349075,10,کیف باشگاه پوما ا puma puma


In [16]:
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL/11.project3/title_categorizer/data/y_test.csv')

In [6]:
# preparing and applying text vectorization to convert clean text into numeric tensor format
vectorizer = keras.layers.TextVectorization()

vectorizer.adapt(train_data['clean_name'].values)

train_data_vectors = vectorizer(train_data['clean_name'].values)
test_data_vectors = vectorizer(test_data['clean_name'].values)

print("Example of transformed train data:", train_data_vectors[0])

Example of transformed train data: tf.Tensor(
[   5  243 1538  463  444    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0], shape=(93,), dtype=int64)


In [12]:
# building and compiling a GRU-based neural network for multi-class text classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout, GlobalMaxPooling1D

VOCAB_SIZE = len(vectorizer.get_vocabulary())
EMBEDDING_DIM = 50
MAX_LENGTH = 100
BATCH_SIZE = 32

model = Sequential()

model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH))

model.add(GRU(128, return_sequences=True))

model.add(GlobalMaxPooling1D())

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# training the model
history = model.fit(
    train_data_vectors, train_data['cat_id'].values,
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 43s 274ms/step - accuracy: 0.3652 - loss: 2.0920
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 41s 299ms/step - accuracy: 0.8805 - loss: 0.3967
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 78s 273ms/step - accuracy: 0.9548 - loss: 0.1577
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 41s 298ms/step - accuracy: 0.9708 - loss: 0.1048
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 79s 277ms/step - accuracy: 0.9848 - loss: 0.0595
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 322ms/step - accuracy: 0.9868 - loss: 0.0483
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 75s 270ms/step - accuracy: 0.9906 - loss: 0.0364
Epoch 8/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 41s 269ms/step - accuracy: 0.9892 - loss: 0.0394
Epoch 9/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 291ms/step - accuracy: 0.9932 - loss: 0.0263
Epoch 10/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 39s 274ms/step - accuracy: 0.9923 - loss: 0.0253


In [17]:
# evaluating training accuracy
from sklearn.metrics import accuracy_score

predictions = model.predict(train_data_vectors)
predictions = predictions.argmax(axis=-1)

y_train_pred = train_data['cat_id'].values

train_accuracy = accuracy_score(y_train_pred, predictions)
print(f'Accuracy on Training set: {train_accuracy:.4f}')

276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step
Accuracy on Training set: 0.9958


In [18]:
# transforming test texts into numerical vectors using the trained vectorizer
test_texts = test_data['clean_name'].values
test_vectors = vectorizer(test_texts)

In [20]:
# evaluating test accuracy
predictions = model.predict(test_vectors)
predicted_labels = predictions.argmax(axis=-1)

y_test_pred = pd.DataFrame({
    'cat_id': predicted_labels
})

test_accuracy = accuracy_score(y_test['cat_id'], y_test_pred['cat_id'])
print(f'Accuracy on Test set: {test_accuracy:.4f}')

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step
Accuracy on Test set: 0.9517
